In [2]:
import os
import requests
from bs4 import BeautifulSoup
from pypdf import PdfReader
from dotenv import load_dotenv
from IPython.display import Markdown, display
from openai import OpenAI

In [5]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
openai = OpenAI()

In [32]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    def __init__(self, url):
        self.url = url
        print(url)
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No Title Found"
        for irrelevant in soup.body("script", "style", "img", "input"):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

class Resume:
    def __init__(self, resume_path):
        self.pdfReader = PdfReader(resume_path)
        self.text = self.pdfReader.pages[0].extract_text()

In [17]:
system_prompt = "You are a resume checker and have internal knowledge about what the ATS (applicant tracking system) is identifying as needed when observing the uploaded resume"

In [64]:
def user_prompt(website, resume):
    user_prompt = f"You are looking at a website titled: {website.title}"
    user_prompt += "\nPlease use this website to make key adjustments to the uploaded resume on hand. \
    I am the person looking to be hired by this company and would like to be given at least a fair chance to interview\
    Please suggest necessary changes and improvements in list format -- especially within the Projects or Experience sections.\
    Also, please make your suggestions using Markdown.\n\n" 
    user_prompt += resume.text
    return user_prompt
    

In [65]:
def messages_for_user(website, resume):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt(website, resume)}
    ]

In [69]:
def summarize(url):
    website = Website(url)
    resume = Resume('grier_resume.pdf')
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages_for_user(website, resume)
    )
    return response.choices[0].message.content

In [70]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [71]:
display_summary('https://search-careers.gm.com/en/jobs/jr-202500570/data-scientist')

https://search-careers.gm.com/en/jobs/jr-202500570/data-scientist


Here are some suggested changes and improvements to your resume to enhance its effectiveness for the Data Scientist position at General Motors. The focus is on aligning your experience and projects with the relevant skills and qualifications commonly sought in data science roles. 

### Suggested Changes

#### 1. **Objective Statement**
   - Add a brief objective statement at the top of your resume to clarify your career goals and emphasize your interest in the Data Scientist position.

   **Example:**
   ```markdown
   **Objective:** Data-driven software engineer with a passion for leveraging machine learning and analytics to solve complex problems. Seeking to contribute my expertise in data processing and software development to General Motors as a Data Scientist.
   ```

#### 2. **Skills Section**
   - Include a dedicated skills section that highlights relevant data science skills such as data analysis, machine learning, statistical analysis, and visualization tools.
   - Specify your proficiency in Python libraries like Pandas, NumPy, and any machine learning frameworks (e.g., Scikit-learn, TensorFlow).

   **Example:**
   ```markdown
   **Skills:**
   - Data Analysis: Pandas, NumPy
   - Machine Learning: Scikit-learn, TensorFlow
   - Data Visualization: Matplotlib, Seaborn, Tableau
   - Languages: Java, Python, JavaScript
   - Databases: PostgreSQL, MySQL, Firebase
   - Tools: Azure DevOps, Git, Jupyter Notebooks
   ```

#### 3. **Projects Section**
   - Create a dedicated **Projects** section to highlight any relevant data science projects. This can showcase your hands-on experience with data analysis or machine learning.

   **Example:**
   ```markdown
   **Projects:**
   - **Customer Churn Prediction Model:** Developed a machine learning model using Python and Scikit-learn to predict customer churn based on historical usage data, achieving an accuracy of 85%. Visualized results with Matplotlib and Seaborn for stakeholder presentations.
   - **Sales Forecasting Dashboard:** Built an interactive sales forecasting dashboard using Flask and JavaScript, integrating historical sales data with predictive analytics to aid in inventory management decision-making.
   ```

#### 4. **Experience Section Adjustments**
   - Focus on quantifying your achievements and explicitly linking them to data science or analysis, if applicable. Use language that emphasizes results, especially in how you handled data and drew insights.

   **Example Adjustments:**
   ```markdown
   **Software Engineer, General Motors - Austin, TX**  
   *May 2023 - Present*  
   - Enhanced PCAR/PCAT application by integrating a major new data object, performing in-depth data analysis and architectural updates on 9 Java microservices and PostgreSQL schema changes.
   - Developed a Python validation tool using OpenPyXL to automate data validation processes, resulting in a 50% reduction in manual validation time for business partners.
   - Managed Flask Python microservices, enabling data collection and analytics that improved application performance metrics.

   **Lead Software Engineer, General Motors - Austin, TX**  
   *May 2021 - Aug 2023*  
   - Led a team of 5 in developing an application that automated data retrieval processes, saving over 10,000 hours of manual work annually for business partners by leveraging data-driven solutions.
   - Designed and implemented RESTful APIs using Swagger documentation, enabling seamless data integration and access for analytical applications.
   ```

#### 5. **Education Section**
   - Add coursework or relevant projects relating to data science within your degrees, if applicable.

   **Example:**
   ```markdown
   **EDUCATION**  
   Northeastern University, M.S. in Computer Science  
   *Sept 2018 - April 2021*  
   Relevant Coursework: Data Science, Machine Learning, Database Systems, Statistical Methods

   California Maritime Academy, B.S. in Business Administration  
   *Apr 2018*
   ```

### Final Thoughts
Ensure that your resume is visually clean, with consistent formatting. Use bullet points for accomplishments and maintain a focus on data-related tasks where possible. Tailoring your resume in this way will help to align your experience with the Data Scientist role at General Motors and improve your chances of landing an interview.